# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
print 'total number of data', len(all_data)

total number of data 223


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:

train_dataset = []
train_targetset = []
for index in permutation:
    train_dataset.append(all_data[index])
    train_targetset.append(all_target[index])
    

clf.fit(train_dataset[:n_training_data], train_targetset[:n_training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
dataset = []
targetset = []
for i in permutation:
    dataset.append(all_data[i])
    targetset.append(all_target[i])
    
print permutation
clf.fit(all_data[:n_training_data], all_target[:n_training_data])

[ 47  79   8 138  73  64  49 211  27 123 110 122  13  33 125 136 183 195
 214 127 152 156  83 169 116  88 133 194  39  17  84  54 203 221 192  23
  38 130 135 205  69 131 144  86 104 158  80 157 175 176  30  85 166  72
  70  24 181  34 170 113 129  29 137  43  77 216  50 103 148  92 204 171
 220 164  74  42 202  71  21 186  11 141   1  53 198 106  93 132 160  65
  46 167 140  58 213  75 199   2 105 119  67 209  31  95 215 210 208  96
 187  44  61 117  18 101 143 173  60  57 120  66  37 154 168 222  59 180
 111 178 188   3  99  52 139  26 134  78  35  63 185 163  36  15 114  81
  82  87 189 149 191 161 196 100 159   0 126 177  19  62 108  48  22 153
 155 174 206  98 118 219 212 193 121 197 142  14   4  94  10  28 218 190
 165 179   6 128  12  40  97 124  41  25 102 109  56  76 201 182  32  51
  89  68 150 200 147 172 107   9   7  16 112 162 184  55  90 217 207  20
 146   5 145  45  91 151 115]


SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [9]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([2]), [10])

In [10]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [11]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data[:n_training_data])
expected = all_target[:n_training_data]

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00        19
           2       0.97      1.00      0.98        30
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        20
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        22

   micro avg       0.99      0.99      0.99       156
   macro avg       1.00      0.99      0.99       156
weighted avg       0.99      0.99      0.99       156


Confusion matrix:
[[24  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0]
 [ 0  0  0  0  0  0 20  0  0]
 [ 0  0  1  0  0  0  0 10  0]
 [ 0  0  0  0

## 4. Evaluate on the test data

In [12]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data[n_training_data:])
expected = all_target[n_training_data:]

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           8       1.00      1.00      1.00         4
           9       0.00      0.00      0.00        11
          10       0.00      0.00      0.00        52

   micro avg       0.06      0.06      0.06        67
   macro avg       0.20      0.20      0.20        67
weighted avg       0.06      0.06      0.06        67


Confusion matrix:
[[ 0  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  4  0  0]
 [ 0 11  0  0  0]
 [50  2  0  0  0]]


E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
E:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [13]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [14]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([2]), [10])